# BikeEase AI Advertisement Generator 🚲

This notebook explores how to use Generative AI (LLMs + LangChain)  
to automatically create marketing advertisements for BikeEase.

**Goals**
- Accept bike specifications, discounts, and themes as input  
- Generate persuasive ad copy with Hugging Face models  
- Use LangChain for structured prompt management


In [2]:
!pip install langchain transformers huggingface_hub torch langchain-community --quiet

from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFacePipeline




In [21]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline



# GPT-2 Small (very lightweight, but not instruction-tuned)
# generator = pipeline(
#     "text-generation",
#     model="distilgpt2",
#     max_new_tokens=150,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.9
# )
# Notes: Fast, but tends to echo prompts. Good for quick sanity checks.

# Dolly v2 3B (instruction-tuned, high quality, heavy)
generator = pipeline(
    "text-generation",
    model="databricks/dolly-v2-3b",
    revision="main",
    trust_remote_code=True,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9
)
# Notes: ~6GB, very slow locally. Better quality, supports instructions.



# GPT-Neo 125M (larger than GPT-2, open-source)
# generator = pipeline(
#     "text-generation",
#     model="EleutherAI/gpt-neo-125M",
#     max_new_tokens=150,
#     temperature=0.7,
#     top_p=0.9
# )
# Notes: More coherent than GPT-2, but not instruction-tuned.



llm = HuggingFacePipeline(pipeline=generator)
print(f"✅ Model loaded: {generator.model.name_or_path}")




Device set to use mps:0


✅ Model loaded: databricks/dolly-v2-3b


In [22]:
ad_prompt = PromptTemplate(
    input_variables=["length", "bike_specs", "discount", "tone", "target_audience"],
    template="""You are a marketing assistant for BikeEase.
    Write a {length} advertisement for a {bike_specs}.
    Highlight the offer: {discount}.
    Use a {tone} tone to appeal to {target_audience}.
    Ensure the content is persuasive and consistent with BikeEase’s branding."""
)


In [23]:
chain = LLMChain(llm=llm, prompt=ad_prompt)

result = chain.run({
    "length": "short social media post",
    "bike_specs": "24-gear mountain bike",
    "discount": "20% off this weekend",
    "tone": "energetic",
    "target_audience": "young adventure seekers"
})

print("Generated Advertisement:\n")
print(result)


Generated Advertisement:

We’re offering 20% off all 24-gear mountain bikes this weekend at BikeEase. This offer is valid online and in-store at our store in your city. Click the link in our bio to see our full lineup of mountain bikes.

Use a hashtag #BikeEase20 on social media to share this post.


In [20]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=120,
    temperature=0.7,
    top_p=0.9
)

llm = HuggingFacePipeline(pipeline=generator)

ad_prompt = PromptTemplate(
    input_variables=["bike_specs", "discount", "tone", "target_audience"],
    template="""You are a creative copywriter for BikeEase. 
Generate short, catchy social media ads. 

Here are some examples:

Example 1:
🚴‍♂️ Adventure awaits! Grab your BikeEase city cruiser 🌆 and enjoy the ride.  
🔥 Limited Time: 10% OFF this weekend!  
#BikeEase #CityLife #RideMore

Example 2:
🌲 Conquer the outdoors with BikeEase’s rugged mountain bikes!  
🌟 First ride special: 15% OFF today only!  
#AdventureAwaits #BikeEase #TrailBlazers

Now write a new ad:

Bike: {bike_specs}  
Offer: {discount}  
Tone: {tone}  
Audience: {target_audience}
"""
)

chain = LLMChain(llm=llm, prompt=ad_prompt)

result = chain.invoke({
    "bike_specs": "24-gear mountain bike",
    "discount": "20% off this weekend",
    "tone": "energetic",
    "target_audience": "young adventure seekers"
})

print("Generated Advertisement:\n")
print(result["text"])



Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated Advertisement:

Bike: 24-gear mountain bike offer: 20% off this weekend


In [25]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Use Dolly v2 3B
generator = pipeline(
    "text-generation",
    model="databricks/dolly-v2-3b",
    revision="main",
    trust_remote_code=True,
    max_new_tokens=250,   # increase so it doesn't truncate
    temperature=0.9,      # a little more creativity
    top_p=0.95,           # wider nucleus sampling
    do_sample=True        # make sure it samples instead of greedy
)

llm = HuggingFacePipeline(pipeline=generator)

# Few-shot prompt to guide Dolly's style
ad_prompt = PromptTemplate(
    input_variables=["bike_specs", "discount", "tone", "target_audience"],
    template="""You are a creative marketing assistant for BikeEase.

Here are examples of BikeEase ads:

Example 1:
🚴‍♂️ Adventure awaits! Grab your BikeEase city cruiser 🌆 and enjoy the ride.  
🔥 Limited Time: 10% OFF this weekend!  
#BikeEase #CityLife #RideMore

Example 2:
🌲 Conquer the outdoors with BikeEase’s rugged mountain bikes!  
🌟 First ride special: 15% OFF today only!  
#AdventureAwaits #BikeEase #TrailBlazers

Now write a {tone} social media ad:

Bike: {bike_specs}  
Offer: {discount}  
Audience: {target_audience}

Keep it under 60 words. Use emojis and hashtags. End with a strong call-to-action.
"""
)

chain = LLMChain(llm=llm, prompt=ad_prompt)

result = chain.invoke({
    "bike_specs": "24-gear mountain bike",
    "discount": "20% off this weekend",
    "tone": "energetic",
    "target_audience": "young adventure seekers"
})

print("Generated Advertisement:\n")
print(result["text"])


Device set to use mps:0


Generated Advertisement:

#bikeease #citylife #ridemore
